**Forecasting Copper Prices**
=============================

In [1]:
! pip install OleFileIO_PL

## **Importing Packages**

In [2]:
import numpy as np
import pandas as pd
import os
import datetime
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import requests
import IPython.display
import OleFileIO_PL
import re

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

## **Loading Datasets**

### Copper Prices

In [3]:
df_copper = pd.read_csv("/content/Copper Futures Historical Data.csv")
df_copper.head(10)

,Date,Price,Open,High,Low,Vol.,Change %
0,10/06/2023,3.6275,3.5660,3.6450,3.5515,NaN,2.13%
1,10/05/2023,3.5520,3.5830,3.6030,3.5490,58.69K,-1.03%
2,10/04/2023,3.5890,3.6185,3.6220,3.5490,76.86K,-0.88%
3,10/03/2023,3.6210,3.6480,3.6480,3.5815,69.45K,-0.56%
4,10/02/2023,3.6415,3.7540,3.7615,3.6275,74.92K,-2.57%
5,09/29/2023,3.7375,3.7125,3.7860,3.7115,82.33K,0.80%
6,09/28/2023,3.7080,3.6465,3.7150,3.6255,73.77K,1.97%
7,09/27/2023,3.6365,3.6465,3.6535,3.6240,64.25K,-0.34%
8,09/26/2023,3.6490,3.6730,3.6835,3.6275,65.80K,-0.57%
9,09/25/2023,3.6700,3.6955,3.7055,3.6490,54.42K,-0.70%


In [4]:
df_copper[["Month", "Day", "Year"]] = df_copper["Date"].str.split("/", expand = True)
df_copper.head()

,Date,Price,Open,High,Low,Vol.,Change %,Month,Day,Year
0,10/06/2023,3.6275,3.5660,3.6450,3.5515,NaN,2.13%,10,06,2023
1,10/05/2023,3.5520,3.5830,3.6030,3.5490,58.69K,-1.03%,10,05,2023
2,10/04/2023,3.5890,3.6185,3.6220,3.5490,76.86K,-0.88%,10,04,2023
3,10/03/2023,3.6210,3.6480,3.6480,3.5815,69.45K,-0.56%,10,03,2023
4,10/02/2023,3.6415,3.7540,3.7615,3.6275,74.92K,-2.57%,10,02,2023


In [5]:
# Remove non-numeric characters from the 'Vol.' column
df_copper['Vol.'] = df_copper['Vol.'].apply(lambda x: re.sub('[^0-9.]', '', str(x)))

# Convert the 'Vol.' column from string to float, handling empty strings
df_copper['Vol.'] = df_copper['Vol.'].apply(lambda x: float(x) if x else float('nan'))

# Print the dataframe to verify the data type conversion
print(df_copper['Vol.'].dtypes)

float64


In [6]:
df_copper['Date'] = pd.to_datetime(df_copper['Date'], errors='coerce')
df_copper

,Date,Price,Open,High,Low,Vol.,Change %,Month,Day,Year
0,2023-10-06,3.6275,3.5660,3.6450,3.5515,NaN,2.13%,10,06,2023
1,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,-1.03%,10,05,2023
2,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,-0.88%,10,04,2023
3,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,-0.56%,10,03,2023
4,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,-2.57%,10,02,2023
...,...,...,...,...,...,...,...,...,...,...
2778,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,-0.03%,01,07,2013
2779,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,-0.39%,01,06,2013
2780,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,-0.63%,01,04,2013
2781,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,-0.51%,01,03,2013


In [7]:
# Remove non-numeric characters from the 'Change %' column
df_copper['Change %'] = df_copper['Change %'].apply(lambda x: re.sub('[^0-9.]', '', str(x)))

# Convert the 'Change %' column from string to float, handling empty strings
df_copper['Change %'] = df_copper['Change %'].apply(lambda x: float(x) if x else float('nan'))
df_copper.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Vol.               float64
Change %           float64
Month               object
Day                 object
Year                object
dtype: object

In [8]:
for i in df_copper:
    if i!="Date":
        df_copper[i]=df_copper[i].astype('float64')
        print(df_copper[i])
    else:
        pass

0       3.6275
1       3.5520
2       3.5890
3       3.6210
4       3.6415
         ...  
2778    3.6780
2779    3.6790
2780    3.6935
2781    3.7170
2782    3.7360
Name: Price, Length: 2783, dtype: float64
0       3.5660
1       3.5830
2       3.6185
3       3.6480
4       3.7540
         ...  
2778    3.7035
2779    3.6840
2780    3.6935
2781    3.7315
2782    3.6510
Name: Open, Length: 2783, dtype: float64
0       3.6450
1       3.6030
2       3.6220
3       3.6480
4       3.7615
         ...  
2778    3.7135
2779    3.6900
2780    3.7070
2781    3.7590
2782    3.7590
Name: High, Length: 2783, dtype: float64
0       3.5515
1       3.5490
2       3.5490
3       3.5815
4       3.6275
         ...  
2778    3.6525
2779    3.6710
2780    3.6710
2781    3.6900
2782    3.6440
Name: Low, Length: 2783, dtype: float64
0         NaN
1       58.69
2       76.86
3       69.45
4       74.92
        ...  
2778    39.01
2779     0.29
2780    45.82
2781    44.09
2782    48.42
Name: Vol., Length: 27

In [9]:
# Remove rows containing null values
df_copper = df_copper.dropna(how='any')
df_copper.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2707 entries, 1 to 2782
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2707 non-null   datetime64[ns]
 1   Price     2707 non-null   float64       
 2   Open      2707 non-null   float64       
 3   High      2707 non-null   float64       
 4   Low       2707 non-null   float64       
 5   Vol.      2707 non-null   float64       
 6   Change %  2707 non-null   float64       
 7   Month     2707 non-null   float64       
 8   Day       2707 non-null   float64       
 9   Year      2707 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 232.6 KB


### Real GDP

In [10]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=REAL_GDP&interval=annual&apikey=demo'
r = requests.get(url)
data = r.json()

# Extract the time series data
time_series = data['data']

# Create a DataFrame from the time series data
df_GDP = pd.DataFrame(time_series)

In [11]:
IPython.display.display(df_GDP)

,date,value
0,2022-01-01,21822.037
1,2021-01-01,21407.692
2,2020-01-01,20234.074
3,2019-01-01,20692.087
4,2018-01-01,20193.896
...,...,...
89,1933-01-01,877.431
90,1932-01-01,888.414
91,1931-01-01,1019.977
92,1930-01-01,1089.785


In [12]:
# Split the "date" column and extract the year
df_GDP['Year'] = df_GDP['date'].str.split('-').str[0]

# Convert the "Year" column to an integer if needed
df_GDP['Year'] = df_GDP['Year'].astype(int)

df_GDP.head()

,date,value,Year
0,2022-01-01,21822.037,2022
1,2021-01-01,21407.692,2021
2,2020-01-01,20234.074,2020
3,2019-01-01,20692.087,2019
4,2018-01-01,20193.896,2018


In [13]:
df_GDP['date'] = pd.to_datetime(df_GDP['date'], errors='coerce')
df_GDP

,date,value,Year
0,2022-01-01,21822.037,2022
1,2021-01-01,21407.692,2021
2,2020-01-01,20234.074,2020
3,2019-01-01,20692.087,2019
4,2018-01-01,20193.896,2018
...,...,...,...
89,1933-01-01,877.431,1933
90,1932-01-01,888.414,1932
91,1931-01-01,1019.977,1931
92,1930-01-01,1089.785,1930


In [14]:
empty = df_GDP[df_GDP['value'] == ''].index
df_GDP.drop(empty , inplace=True)

In [15]:
df_GDP['value'] = df_GDP['value'].astype(float)

In [16]:
df_GDP.rename(columns = {'value':'Real_GDP'}, inplace = True)
df_GDP

,date,Real_GDP,Year
0,2022-01-01,21822.037,2022
1,2021-01-01,21407.692,2021
2,2020-01-01,20234.074,2020
3,2019-01-01,20692.087,2019
4,2018-01-01,20193.896,2018
...,...,...,...
89,1933-01-01,877.431,1933
90,1932-01-01,888.414,1932
91,1931-01-01,1019.977,1931
92,1930-01-01,1089.785,1930


In [17]:
df_GDP = df_GDP[df_GDP['date'] >= '2013-01-01']
df_GDP

,date,Real_GDP,Year
0,2022-01-01,21822.037,2022
1,2021-01-01,21407.692,2021
2,2020-01-01,20234.074,2020
3,2019-01-01,20692.087,2019
4,2018-01-01,20193.896,2018
5,2017-01-01,19612.102,2017
6,2016-01-01,19141.672,2016
7,2015-01-01,18799.622,2015
8,2014-01-01,18261.714,2014
9,2013-01-01,17812.167,2013


### Consumer Price Index (CPI)

In [18]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data_CPI = r.json()

# Extract the time series data
time_series_CPI = data_CPI['data']

# Create a DataFrame from the time series data
df_CPI = pd.DataFrame(time_series_CPI)

In [19]:
IPython.display.display(df_CPI)

,date,value
0,2023-08-01,307.026
1,2023-07-01,305.691
2,2023-06-01,305.109
3,2023-05-01,304.127
4,2023-04-01,303.363
...,...,...
1323,1913-05-01,9.700
1324,1913-04-01,9.800
1325,1913-03-01,9.800
1326,1913-02-01,9.800


In [20]:
# # Split the "date" column and extract the year
# df_CPI['Year'] = df_CPI['date'].str.split('-').str[0]
# df_CPI['Month'] = df_CPI['date'].str.split('-').str[1]

# # Convert the "Year" column to an integer if needed
# df_CPI['Year'] = df_CPI['Year'].astype(int)
# df_CPI['Month'] = df_CPI['Month'].astype(int)

# df_CPI.head()

In [21]:
df_CPI['date'] = pd.to_datetime(df_CPI['date'], errors='coerce')
df_CPI

,date,value
0,2023-08-01,307.026
1,2023-07-01,305.691
2,2023-06-01,305.109
3,2023-05-01,304.127
4,2023-04-01,303.363
...,...,...
1323,1913-05-01,9.700
1324,1913-04-01,9.800
1325,1913-03-01,9.800
1326,1913-02-01,9.800


In [22]:
empty_CPI = df_CPI[df_CPI['value'] == ''].index
df_CPI.drop(empty_CPI , inplace=True)

In [23]:
df_CPI['value'] = df_CPI['value'].astype(float)
df_CPI.dtypes

date     datetime64[ns]
value           float64
dtype: object

In [24]:
df_CPI.rename(columns = {'value':'CPI', 'date':'Date'}, inplace = True)
df_CPI

,Date,CPI
0,2023-08-01,307.026
1,2023-07-01,305.691
2,2023-06-01,305.109
3,2023-05-01,304.127
4,2023-04-01,303.363
...,...,...
1323,1913-05-01,9.700
1324,1913-04-01,9.800
1325,1913-03-01,9.800
1326,1913-02-01,9.800


In [25]:
df_CPI = df_CPI[df_CPI['Date'] >= '2013-01-01']
df_CPI

,Date,CPI
0,2023-08-01,307.026
1,2023-07-01,305.691
2,2023-06-01,305.109
3,2023-05-01,304.127
4,2023-04-01,303.363
...,...,...
123,2013-05-01,232.945
124,2013-04-01,232.531
125,2013-03-01,232.773
126,2013-02-01,232.166


### Inflation Rates

In [26]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=INFLATION&apikey=demo'
r = requests.get(url)
inflation = r.json()

# Extract the time series data
time_series = inflation['data']

# Create a DataFrame from the time series data
df_inf = pd.DataFrame(time_series)

In [27]:
df_inf.head(10)

,date,value
0,2022-01-01,8.00279982052117
1,2021-01-01,4.69785886363739
2,2020-01-01,1.23358439630637
3,2019-01-01,1.81221007526015
4,2018-01-01,2.44258329692818
5,2017-01-01,2.13011000365963
6,2016-01-01,1.26158320570537
7,2015-01-01,0.118627135552435
8,2014-01-01,1.62222297740821
9,2013-01-01,1.46483265562714


In [28]:
df_inf.rename(columns = {'value':'inflation_rate','date':'Date'}, inplace = True)
df_inf

,Date,inflation_rate
0,2022-01-01,8.00279982052117
1,2021-01-01,4.69785886363739
2,2020-01-01,1.23358439630637
3,2019-01-01,1.81221007526015
4,2018-01-01,2.44258329692818
...,...,...
58,1964-01-01,1.27891156462583
59,1963-01-01,1.2396694214876
60,1962-01-01,1.19877334820185
61,1961-01-01,1.07072414764723


In [29]:
# Split the "date" column and extract the year
df_inf['Year'] = df_inf['Date'].str.split('-').str[0]

# Convert the "Year" column to an integer if needed
df_inf['Year'] = df_inf['Year'].astype(int)

df_inf.head()

,Date,inflation_rate,Year
0,2022-01-01,8.00279982052117,2022
1,2021-01-01,4.69785886363739,2021
2,2020-01-01,1.23358439630637,2020
3,2019-01-01,1.81221007526015,2019
4,2018-01-01,2.44258329692818,2018


In [30]:
df_inf['Date'] = pd.to_datetime(df_inf['Date'], errors='coerce')
df_inf.dtypes

Date              datetime64[ns]
inflation_rate            object
Year                       int64
dtype: object

In [31]:
df_inf = df_inf[df_inf['Date'] >= '2013-01-01']
df_inf

,Date,inflation_rate,Year
0,2022-01-01,8.00279982052117,2022
1,2021-01-01,4.69785886363739,2021
2,2020-01-01,1.23358439630637,2020
3,2019-01-01,1.81221007526015,2019
4,2018-01-01,2.44258329692818,2018
5,2017-01-01,2.13011000365963,2017
6,2016-01-01,1.26158320570537,2016
7,2015-01-01,0.118627135552435,2015
8,2014-01-01,1.62222297740821,2014
9,2013-01-01,1.46483265562714,2013


### Global Price Index of All Commodities

In [32]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=ALL_COMMODITIES&interval=monthly&apikey=demo'
r = requests.get(url)
com = r.json()

# Extract the time series data
time_series = com['data']

# Create a DataFrame from the time series data
df_com = pd.DataFrame(time_series)
df_com

,date,value
0,2023-08-01,161.451785068472
1,2023-07-01,157.948024050167
2,2023-06-01,154.002090208084
3,2023-05-01,157.126974621549
4,2023-04-01,170.764655371124
...,...,...
375,1992-05-01,.
376,1992-04-01,.
377,1992-03-01,.
378,1992-02-01,.


In [33]:
df_com.rename(columns = {'value':'PALLFNFINDEXM','date':'Date'}, inplace = True)

df_com['Date'] = pd.to_datetime(df_com['Date'], errors='coerce')
df_com.dtypes

Date             datetime64[ns]
PALLFNFINDEXM            object
dtype: object

In [34]:
df_com = df_com[df_com['Date'] >= '2013-01-01']
df_com

,Date,PALLFNFINDEXM
0,2023-08-01,161.451785068472
1,2023-07-01,157.948024050167
2,2023-06-01,154.002090208084
3,2023-05-01,157.126974621549
4,2023-04-01,170.764655371124
...,...,...
123,2013-05-01,166.74340866391
124,2013-04-01,167.819250104005
125,2013-03-01,171.203032018493
126,2013-02-01,175.50395775166


In [35]:
df_com['PALLFNFINDEXM'] = df_com['PALLFNFINDEXM'].astype(float)
df_com.dtypes

Date             datetime64[ns]
PALLFNFINDEXM           float64
dtype: object

## **Merging Datasets**

In [36]:
# Merge data frames based on a common key
merged_df_1 = pd.merge(df_copper, df_GDP, left_on='Year', right_on='Year', how='outer')
merged_df_1

,Date,Price,Open,High,Low,Vol.,Change %,Month,Day,Year,date,Real_GDP
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,5.0,2023.0,NaT,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,4.0,2023.0,NaT,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,3.0,2023.0,NaT,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2.0,2023.0,NaT,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,29.0,2023.0,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,7.0,2013.0,2013-01-01,17812.167
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,6.0,2013.0,2013-01-01,17812.167
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,4.0,2013.0,2013-01-01,17812.167
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,3.0,2013.0,2013-01-01,17812.167


In [37]:
merged_df_1 = merged_df_1.drop(['date','Day'], axis=1)
merged_df_1

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167


In [38]:
# Forward fill the missing Real_GDP values by grouping them by Year
merged_df_1['Real_GDP'] = merged_df_1.groupby('Year')['Real_GDP'].fillna(method='ffill')
merged_df_1

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167


> Real_GDP value change every year

In [39]:
# Set a constant value for missing 'Real_GDP' in the year 2023
constant_value_2023 = 19374

# Fill the missing 'Real_GDP' values with the constant value for 2023
merged_df_1['Real_GDP'] = merged_df_1['Real_GDP'].fillna(
    merged_df_1['Year'].apply(lambda x: constant_value_2023 if x == 2023 else np.nan))

In [40]:
merged_df_1

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000
...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167


In [41]:
# Interpolate the missing values using linear interpolation
merged_df_1.interpolate(method='ffill', inplace=True)
merged_df_1

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000
...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167


In [42]:
merged_df_2 = pd.merge(merged_df_1, df_CPI, on='Date', how='left')
merged_df_2
# print(merged_df_2['CPI'].nunique())

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,NaN
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,NaN
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,NaN
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,NaN


In [43]:
merged_df_2.isnull().sum()

Date           0
Price          0
Open           0
High           0
Low            0
Vol.           0
Change %       0
Month          0
Year           0
Real_GDP       0
CPI         2624
dtype: int64

In [44]:
# Group by year and month, and transform Indicator1 to fill missing values with the first value of the group
merged_df_2['CPI'] = merged_df_2.groupby([merged_df_2['Date'].dt.year,
                                                 merged_df_2['Date'].dt.month])['CPI'].transform('first')

merged_df_2['CPI'] = merged_df_2['CPI'].ffill()
merged_df_2

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166


In [45]:
# Merge data frames based on a common key
merged_df_3 = pd.merge(merged_df_2, df_inf, left_on='Year', right_on='Year', how='outer')
merged_df_3

,Date_x,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,Date_y,inflation_rate
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,NaT,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,NaT,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,NaT,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,NaT,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,2013-01-01,1.46483265562714
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,2013-01-01,1.46483265562714
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,2013-01-01,1.46483265562714
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,2013-01-01,1.46483265562714


In [46]:
merged_df_3 = merged_df_3.drop(['Date_y'], axis=1)
# merged_df_3 = merged_df_3.rename({'Date_x': 'Date'}, inplace=True)
merged_df_3.rename(columns = {'Date_x': 'Date'}, inplace = True)
merged_df_3

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,inflation_rate
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,1.46483265562714
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,1.46483265562714
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,1.46483265562714
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,1.46483265562714


In [47]:
# Forward fill the missing Real_GDP values by grouping them by Year
merged_df_3['inflation_rate'] = merged_df_3.groupby('Year')['inflation_rate'].fillna(method='ffill')
merged_df_3

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,inflation_rate
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,1.46483265562714
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,1.46483265562714
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,1.46483265562714
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,1.46483265562714


In [48]:
# Set a constant value for missing 'inflation_rate' in the year 2023
inflation_rate_2023 = 6.6

# Fill the missing 'inflation_rate' values with the constant value for 2023
merged_df_3['inflation_rate'] = merged_df_3['inflation_rate'].fillna(
    merged_df_3['Year'].apply(lambda x: inflation_rate_2023 if x == 2023 else np.nan))

merged_df_3

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,inflation_rate
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,6.6
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,6.6
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,6.6
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,6.6
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,1.46483265562714
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,1.46483265562714
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,1.46483265562714
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,1.46483265562714


In [49]:
final_merged_dataset = pd.merge(merged_df_3, df_com, on='Date', how='left')
final_merged_dataset

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,6.6,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,6.6,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,6.6,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,6.6,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,6.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,1.46483265562714,NaN
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,1.46483265562714,NaN
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,1.46483265562714,NaN
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,1.46483265562714,NaN


In [50]:
final_merged_dataset['PALLFNFINDEXM'].isnull().sum()

2624

In [51]:
# Group by year and month, and transform Indicator1 to fill missing values with the first value of the group
final_merged_dataset['PALLFNFINDEXM'] = final_merged_dataset.groupby([final_merged_dataset['Date'].dt.year,
                                                 final_merged_dataset['Date'].dt.month])['CPI'].transform('first')

final_merged_dataset['PALLFNFINDEXM'] = final_merged_dataset['PALLFNFINDEXM'].ffill()
final_merged_dataset

,Date,Price,Open,High,Low,Vol.,Change %,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2023-10-05,3.5520,3.5830,3.6030,3.5490,58.69,1.03,10.0,2023.0,19374.000,NaN,6.6,NaN
1,2023-10-04,3.5890,3.6185,3.6220,3.5490,76.86,0.88,10.0,2023.0,19374.000,NaN,6.6,NaN
2,2023-10-03,3.6210,3.6480,3.6480,3.5815,69.45,0.56,10.0,2023.0,19374.000,NaN,6.6,NaN
3,2023-10-02,3.6415,3.7540,3.7615,3.6275,74.92,2.57,10.0,2023.0,19374.000,NaN,6.6,NaN
4,2023-09-29,3.7375,3.7125,3.7860,3.7115,82.33,0.80,9.0,2023.0,19374.000,NaN,6.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2013-01-07,3.6780,3.7035,3.7135,3.6525,39.01,0.03,1.0,2013.0,17812.167,232.166,1.46483265562714,232.166
2703,2013-01-06,3.6790,3.6840,3.6900,3.6710,0.29,0.39,1.0,2013.0,17812.167,232.166,1.46483265562714,232.166
2704,2013-01-04,3.6935,3.6935,3.7070,3.6710,45.82,0.63,1.0,2013.0,17812.167,232.166,1.46483265562714,232.166
2705,2013-01-03,3.7170,3.7315,3.7590,3.6900,44.09,0.51,1.0,2013.0,17812.167,232.166,1.46483265562714,232.166


In [52]:
final_merged_dataset['PALLFNFINDEXM'].isnull().sum()

24

In [53]:
final_merged_dataset.to_csv('final_merged_copper_dataset_new.csv')